In [14]:
# readin data
win_dir = "..\data\\"
unix_dir = "../data/"
if(isdir(win_dir))
    data_dir = win_dir
elseif(isdir(unix_dir))
    data_dir = unix_dir
end
print(data_dir)

#cost of transhipment over boat. Takes one month
ocean_costs = readdlm(data_dir*"ocean_cost.tsv", '\t')
temp = zeros()
#cost of transhipment over air. Instant
air_costs = readdlm(data_dir*"air_cost.tsv", '\t')

#costs to hold a boat at non retail location
holding_costs = readdlm(data_dir*"holding_cost.tsv",'\t')
risk_costs = readdlm(data_dir*"risk_cost.tsv", '\t')
demand_7536 = readdlm(data_dir*"demand_7536.tsv",'\t')
demand_7878 = readdlm(data_dir*"demand_7878.tsv",'\t')
demand_7585 = readdlm(data_dir*"demand_7585.tsv",'\t')


../data/

12×6 Array{Float64,2}:
 110.0  168.0   90.0   86.0  50.0  45.0
 131.5  174.0  130.0   90.0  68.0  60.0
 169.5  195.0  175.0   96.0  70.0  60.0
 195.0  213.0  195.0  106.0  70.0  60.0
 180.0  235.0  178.0  120.0  87.0  75.0
 165.0  260.0  162.0  130.0  90.0  70.0
 155.0  241.0  152.0  124.0  78.0  65.0
 145.0  210.0  146.0  105.0  77.0  65.0
 132.5  189.0  130.0   94.0  60.0  65.0
 138.5  165.0  136.0   82.0  60.0  65.0
 122.5  160.0  120.0   80.0  42.0  50.0
 102.5  158.0  100.0   80.0  40.0  50.0

In [24]:
lambda = 0
net_profit_7536 = 100*ones(6)
net_profit_7878 = 100*ones(6)
net_profit_7585 = 100*ones(6)

cur_product = "7536"

"7536"

In [29]:
# Select current profit/boat at each retailer
if cur_product == "7536"
    net_profit = net_profit_7536
elseif cur_product == "7878"
    net_profit = net_profit_7878
elseif cur_product == "7585"
    net_profit = net_profit_7585
end

#define an inverse incidence tensor for our flow (0 implies incidence)
inv_incidence_tensor = ones(12,21,12,21)

#cost of flows
cost_tensor = zeros(12,21,12,21)

#Expedite edges
for t in 1:12
    #1 can expedite to its hubs
    inv_incidence_tensor[t,1,t,2:3] = 0
    cost_tensor[t,1,t,2:3] = air_costs[1,2:3]
    
    
    #2 can expedite to its dc's
    inv_incidence_tensor[t,2,t,4:6] = 0
    cost_tensor[t,2,t,4:6] = air_costs[2,4:6]
    
    #3 can expedite to its dc's
    inv_incidence_tensor[t,3,t,7:9] = 0
    cost_tensor[t,3,t,7:9] = air_costs[3,7:9]
    
    #Some Dc's can expedite to eachother
    inv_incidence_tensor[t,4,t,5] = 0
    cost_tensor[t,4,t,5] = air_costs[4,5]
    inv_incidence_tensor[t,5,t,4] = 0
    cost_tensor[t,5,t,4] = air_costs[5,4]
    inv_incidence_tensor[t,7,t,8] = 0
    cost_tensor[t,7,t,8] = air_costs[7,8]
    inv_incidence_tensor[t,8,t,7] = 0
    cost_tensor[t,8,t,7] = air_costs[8,7]
    inv_incidence_tensor[t,8,t,9] = 0
    cost_tensor[t,8,t,9] = air_costs[8,9]
    inv_incidence_tensor[t,9,t,8] = 0
    cost_tensor[t,9,t,8] = air_costs[9,8]
    
    #Dc's can expedite to retailers
    for i in 4:9 
        inv_incidence_tensor[t,i,t,i+6] = 0
        cost_tensor[t,i,t,i+6] = air_costs[i,i+6] - net_profit[i-3]
    end
end

#Shipping edges
for t in 1:11
    #1 can ship to its hubs
    inv_incidence_tensor[t,1,t+1,2:3] = 0
    cost_tensor[t,1,t+1,2:3] = ocean_costs[1,2:3]
    
    
    #2 can ship to its dc's
    inv_incidence_tensor[t,2,t+1,4:6] = 0
    cost_tensor[t,2,t+1,4:6] = ocean_costs[2,4:6]
    
    #3 can ship to its dc's
    inv_incidence_tensor[t,3,t+1,7:9] = 0
    cost_tensor[t,3,t+1,7:9] = ocean_costs[3,7:9]
    
    #Some Dc's can ship to eachother
    inv_incidence_tensor[t,4,t+1,5] = 0
    cost_tensor[t,4,t+1,5] = ocean_costs[4,5]
    inv_incidence_tensor[t,5,t+1,4] = 0
    cost_tensor[t,5,t+1,4] = ocean_costs[5,4]
    inv_incidence_tensor[t,7,t+1,8] = 0
    cost_tensor[t,7,t+1,8] = ocean_costs[7,8]
    inv_incidence_tensor[t,8,t+1,7] = 0
    cost_tensor[t,8,t+1,7] = ocean_costs[8,7]
    inv_incidence_tensor[t,8,t+1,9] = 0
    cost_tensor[t,8,t+1,9] = ocean_costs[8,9]
    inv_incidence_tensor[t,9,t+1,8] = 0
    cost_tensor[t,9,t+1,8] = ocean_costs[9,8]
    
    #Dc's can ship to retailers
    for i in 4:9 
        inv_incidence_tensor[t,i,t+1,i+6] = 0
        cost_tensor[t,i,t+1,i+6] = ocean_costs[i,i+6] - net_profit[i-3]
    end
end

#Holding edges


In [37]:
using JuMP, Clp
#model
m = Model(solver =ClpSolver())

#Flow tensor
@variable(m, f[1:12,1:21,1:12,1:21] >= 0)


12×21×12×21 Array{JuMP.Variable,4}:
[:, :, 1, 1] =
 f[1,1,1,1]   f[1,2,1,1]   f[1,3,1,1]   …  f[1,20,1,1]   f[1,21,1,1] 
 f[2,1,1,1]   f[2,2,1,1]   f[2,3,1,1]      f[2,20,1,1]   f[2,21,1,1] 
 f[3,1,1,1]   f[3,2,1,1]   f[3,3,1,1]      f[3,20,1,1]   f[3,21,1,1] 
 f[4,1,1,1]   f[4,2,1,1]   f[4,3,1,1]      f[4,20,1,1]   f[4,21,1,1] 
 f[5,1,1,1]   f[5,2,1,1]   f[5,3,1,1]      f[5,20,1,1]   f[5,21,1,1] 
 f[6,1,1,1]   f[6,2,1,1]   f[6,3,1,1]   …  f[6,20,1,1]   f[6,21,1,1] 
 f[7,1,1,1]   f[7,2,1,1]   f[7,3,1,1]      f[7,20,1,1]   f[7,21,1,1] 
 f[8,1,1,1]   f[8,2,1,1]   f[8,3,1,1]      f[8,20,1,1]   f[8,21,1,1] 
 f[9,1,1,1]   f[9,2,1,1]   f[9,3,1,1]      f[9,20,1,1]   f[9,21,1,1] 
 f[10,1,1,1]  f[10,2,1,1]  f[10,3,1,1]     f[10,20,1,1]  f[10,21,1,1]
 f[11,1,1,1]  f[11,2,1,1]  f[11,3,1,1]  …  f[11,20,1,1]  f[11,21,1,1]
 f[12,1,1,1]  f[12,2,1,1]  f[12,3,1,1]     f[12,20,1,1]  f[12,21,1,1]

[:, :, 2, 1] =
 f[1,1,2,1]   f[1,2,2,1]   f[1,3,2,1]   …  f[1,20,2,1]   f[1,21,2,1] 
 f[2,1,2,1]   f[2,2,2,1